In [1]:
import os
import time
from time import perf_counter

import numpy as np

from hybrid_derivative_calculator import HybridDerivativeCalculator
import matplotlib.pyplot as plt


In [6]:
%%time
def run_tests():
    test_cases = [
    {"name": "x^2", "func": lambda x: x**2},
    {"name": "x^3", "func": lambda x: x**3},
    {"name": "sin(x)", "func": lambda x: np.sin(x)},
    {"name": "cos(x)", "func": lambda x: np.cos(x)},
    {"name": "tan(x)", "func": lambda x: np.tan(x)},
    {"name": "exp(x)", "func": lambda x: np.exp(x)},
    {"name": "ln(x + 2)", "func": lambda x: np.log(x + 2)},
    {"name": "sqrt(x + 1)", "func": lambda x: np.sqrt(x + 1)},
    {"name": "1 / (1 + x^2)", "func": lambda x: 1 / (1 + x**2)},
    {"name": "x * sin(x)", "func": lambda x: x * np.sin(x)},
    {"name": "x^2 * cos(x)", "func": lambda x: x**2 * np.cos(x)},
    {"name": "x * exp(-x^2)", "func": lambda x: x * np.exp(-x**2)},
    {"name": "exp(-x^2) * sin(5x)", "func": lambda x: np.exp(-x**2) * np.sin(5 * x)},
    {"name": "log(1 + x^2) * sin(x)", "func": lambda x: np.log(1 + x**2) * np.sin(x)},
    {"name": "sin(x) * cos(x)", "func": lambda x: np.sin(x) * np.cos(x)},
    {"name": "abs(x)", "func": lambda x: np.abs(x)},
    {"name": "Piecewise (x^2 / sqrt(x+1))", "func": lambda x: x**2 if x < 0 else np.sqrt(x + 1)},
    {"name": "Piecewise sin or cos", "func": lambda x: np.sin(x) if x < 1 else np.cos(x)},
    {"name": "ReLU", "func": lambda x: max(0, x)},
    {"name": "x * |x|", "func": lambda x: x * abs(x)},
    {"name": "Heaviside(x)", "func": lambda x: 0 if x < 0 else 1},
    {"name": "sin(x^2)", "func": lambda x: np.sin(x**2)},
    {"name": "exp(sin(x))", "func": lambda x: np.exp(np.sin(x))},
    {"name": "log(cos(x)^2 + 1)", "func": lambda x: np.log(np.cos(x)**2 + 1)},
    {"name": "weird combo", "func": lambda x: x**3 * np.cos(x**2) + np.exp(-x**2)},

    # Extra complicated functions
    {"name": "log(|sin(x)| + 1)", "func": lambda x: np.log(np.abs(np.sin(x)) + 1)},
    {"name": "exp(-x^2) * cos(10x)", "func": lambda x: np.exp(-x**2) * np.cos(10 * x)},
    {"name": "arctan(exp(x))", "func": lambda x: np.arctan(np.exp(x))},
    {"name": "nested trig log", "func": lambda x: np.log(np.abs(np.sin(x)) + 1e-3)},
    {"name": "sqrt(sin^2(x) + cos^2(x))", "func": lambda x: np.sqrt(np.sin(x)**2 + np.cos(x)**2)},
    {"name": "sigmoid", "func": lambda x: 1 / (1 + np.exp(-x))},
    {"name": "softplus", "func": lambda x: np.log(1 + np.exp(x))},
    {"name": "x * tanh(x)", "func": lambda x: x * np.tanh(x)},
    {"name": "gaussian bump", "func": lambda x: np.exp(-1 / (1 - x**2)) if abs(x) < 1 else 0},
    {"name": "x^x (safe)", "func": lambda x: x**x if x > 0 else 0},
    {"name": "sin(exp(x))", "func": lambda x: np.sin(np.exp(x))},
    {"name": "cos(sin(x^2))", "func": lambda x: np.cos(np.sin(x**2))},
    {"name": "sinc(x)", "func": lambda x: np.sinc(x / np.pi)},  # normalized sinc
]

    x_vals = [-16, -5, 0.0, 0.5, 3.6]
    print("\n=== DERIVATIVE TEST RESULTS ===\n")

    for case in test_cases:
        print(f"Function: {case['name']}")
        for x in x_vals:
            try:
                hdc = HybridDerivativeCalculator(case["func"], x, derivative_order=2, log_suffix="run")
                stem = hdc.stem_method()
                stencil = hdc.five_point_stencil_method()
                abs_diff = abs(stem - stencil)
                rel_diff = abs_diff / (abs(stencil) + 1e-12)

                print(f"  @ x = {x:>5.1f} | Stem: {stem:>10.6f} | Stencil: {stencil:>10.6f} | Δ: {abs_diff:>8.2e} | Rel Δ: {rel_diff:>6.2e}", end="")
                if rel_diff > 0.05:
                    print("  ⚠️")
                else:
                    print("  ✅")
            except Exception as e:
                print(f"  @ x = {x:>5.1f} → ❌ Error: {str(e)}")
        print("-" * 80)

if __name__ == "__main__":
    run_tests()



=== DERIVATIVE TEST RESULTS ===

Function: x^2
  @ x = -16.0 | Stem:   2.000000 | Stencil:   2.000000 | Δ: 1.08e-09 | Rel Δ: 5.38e-10  ✅
  @ x =  -5.0 | Stem:   2.000000 | Stencil:   2.000000 | Δ: 5.82e-11 | Rel Δ: 2.91e-11  ✅
  @ x =   0.0 | Stem:   2.000000 | Stencil:   2.000000 | Δ: 1.11e-15 | Rel Δ: 5.55e-16  ✅
  @ x =   0.5 | Stem:   2.000000 | Stencil:   2.000000 | Δ: 3.09e-13 | Rel Δ: 1.55e-13  ✅
  @ x =   3.6 | Stem:   2.000000 | Stencil:   2.000000 | Δ: 1.82e-11 | Rel Δ: 9.10e-12  ✅
--------------------------------------------------------------------------------
Function: x^3
  @ x = -16.0 | Stem: -96.000000 | Stencil: -96.000000 | Δ: 2.37e-08 | Rel Δ: 2.47e-10  ✅
  @ x =  -5.0 | Stem: -30.000000 | Stencil: -30.000000 | Δ: 1.31e-10 | Rel Δ: 4.37e-12  ✅
  @ x =   0.0 | Stem:   0.000000 | Stencil:   0.000000 | Δ: 0.00e+00 | Rel Δ: 0.00e+00  ✅
  @ x =   0.5 | Stem:   3.000000 | Stencil:   3.000000 | Δ: 5.28e-14 | Rel Δ: 1.76e-14  ✅
  @ x =   3.6 | Stem:  21.600000 | Stencil:  21

<timed exec>:9: RuntimeWarning: invalid value encountered in log
<timed exec>:10: RuntimeWarning: invalid value encountered in sqrt


In [ ]:
def plot_function_with_residuals(f, x0, stem_slope, stencil_slope, dx=0.5,
                                 title="Function and Tangents + Residuals",
                                 save_path=None):
    x = np.linspace(x0 - dx, x0 + dx, 400)
    y = f(x)
    f0 = f(x0)

    # Tangents
    tangent_stem = stem_slope * (x - x0) + f0
    tangent_stencil = stencil_slope * (x - x0) + f0

    # Residuals
    residual_stem = y - tangent_stem
    residual_stencil = y - tangent_stencil

    # Set up two vertically stacked subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 7), sharex=True,
                                   gridspec_kw={'height_ratios': [2, 1]})

    # Top panel: function and tangents
    ax1.plot(x, y, label='Function', color='black')
    ax1.plot(x, tangent_stem, lw=2.5, label='Stem Tangent', color='yellowgreen')  # fixed here
    ax1.plot(x, tangent_stencil, '--', lw=2.5, label='Stencil Tangent', color='hotpink')
    ax1.axvline(x0, ls='--', lw=1.5, color='gray', alpha=0.6)
    ax1.set_ylabel("$f(x)$", fontsize=14)
    ax1.legend()
    ax1.set_title(title, fontsize=16)

    # Bottom panel: residuals
    ax2.plot(x, residual_stem, label='Stem Residual', color='yellowgreen', lw=2)
    ax2.plot(x, residual_stencil, label='Stencil Residual', color='hotpink', lw=2, ls="--")
    ax2.axhline(0, ls='--', color='gray', lw=1)
    ax2.set_xlabel("$x$", fontsize=14)
    ax2.set_ylabel("Residual", fontsize=13)
    ax2.legend()

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)


In [ ]:
test_functions = {
    "x_squared": lambda x: x**2,
    "x_cubed": lambda x: x**3,
    "sin_x": lambda x: np.sin(x),
    "cos_x": lambda x: np.cos(x),
    "tan_x": lambda x: np.tan(x),
    "exp_x": lambda x: np.exp(x),
    "log_x_plus_2": lambda x: np.log(x + 2),
    "sqrt_x_plus_1": lambda x: np.sqrt(x + 1),
    "inv_1_plus_x2": lambda x: 1 / (1 + x**2),
    "x_sin_x": lambda x: x * np.sin(x),
    "x2_cos_x": lambda x: x**2 * np.cos(x),
    "x_exp_neg_x2": lambda x: x * np.exp(-x**2),
    "exp_sin5x": lambda x: np.exp(-x**2) * np.sin(5 * x),
    "log1px2_sinx": lambda x: np.log(1 + x**2) * np.sin(x),
    "sinx_cosx": lambda x: np.sin(x) * np.cos(x),
    "abs_x": lambda x: np.abs(x),
    "piecewise_quad_sqrt": lambda x: np.where(x < 0, x**2, np.sqrt(x + 1)),
    "piecewise_sin_cos": lambda x: np.where(x < 1, np.sin(x), np.cos(x)),
    "relu": lambda x: np.maximum(0, x),
    "x_times_absx": lambda x: x * abs(x),
    "heaviside": lambda x: np.where(x < 0, 0, 1),
    "sin_x2": lambda x: np.sin(x**2),
    "exp_sin_x": lambda x: np.exp(np.sin(x)),
    "log_cos2_plus_1": lambda x: np.log(np.cos(x)**2 + 1),
    "weird_combo": lambda x: x**3 * np.cos(x**2) + np.exp(-x**2),
}


In [ ]:
def plot_all_derivatives(func_dict, x_center_list, save_dir="plots_tests"):
    os.makedirs(save_dir, exist_ok=True)

    for name, f in func_dict.items():
        for x0 in x_center_list:
            try:
                hdc = HybridDerivativeCalculator(f, x0, log_suffix=name)
                stem = hdc.stem_method()
                stencil = hdc.five_point_stencil_method()

                # Combined plot with function, tangents, and residuals
                plot_function_with_residuals(
                    f, x0, stem, stencil, dx=0.4,
                    title=f"{name} @ x={x0}",
                    save_path=os.path.join(save_dir, f"{name}_x{x0:.2f}_combined.png")
                )

            except Exception as e:
                print(f"❌ Error for {name} at x={x0}: {e}")


In [ ]:
%%time
x_vals = [-3.6, -0.5, 0.0, 0.5, 3.6]
plot_all_derivatives(test_functions, x_vals)


In [4]:
# Define a computationally heavy function
def slow_function(x):
    time.sleep(0.1)  # simulate expensive evaluation
    return np.sin(x**2) * np.exp(-x**2) + np.log(1 + x**2)

# Initialize the calculator
calc = HybridDerivativeCalculator(slow_function, central_value=0.0005, derivative_order=2)

# Benchmark both methods
for method in ['stem', 'five_point_stencil']:
    start = perf_counter()
    result = calc.stem_method() if method == 'stem' else calc.five_point_stencil_method()
    duration = perf_counter() - start
    print(f"{method.capitalize()} method result: {result:.6f} | Time taken: {duration:.4f} seconds")


Stem method result: 3.999995 | Time taken: 1.9745 seconds
Five_point_stencil method result: 3.999995 | Time taken: 0.6156 seconds
